### <span style="color: #3A6D8C;">Download records according with events.xlsx</span>

In [1]:
import pandas as pd

from pathlib import Path
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.mass_downloader import RectangularDomain, Restrictions, MassDownloader

In [2]:
excel_file = './events/events.xlsx'

df = pd.read_excel(excel_file, sheet_name='info', index_col=None)

# chose 2013-07-20 event as a demo
event_num = 7

# Origin time 
origin_time = UTCDateTime(df.loc[event_num, 'Origin Time (UTC)'])
event_data = '{0:04d}-{1:02d}-{2:02d}'.format(origin_time.year, origin_time.month,origin_time.day)
folder_output = Path(excel_file).absolute().parent.joinpath(event_data)

if not folder_output.exists():
    folder_output.mkdir(parents=True)

# define search geography extent
minlatitude = 42.9
maxlatitude = 53.7
minlongitude = 6.4
maxlongitude = 7.3

#seconds to request before and after the given date and time
second_before = 150
second_after = 300

network = 'NL'
providers = "http://rdsa.knmi.nl"
# requested channels
channel_priorities = ["HG[ZNE12]"]

<div style="color:#88C273; padding: 10px; font-size: 20px"> 
For particular event, firstly setting geographical location,
then preprocessing condition are needed where MassDownloader plays a foundmental role. 
</div>

In [ ]:
domain = RectangularDomain(minlatitude=minlatitude,
                            maxlatitude=maxlatitude,
                            minlongitude=minlongitude,
                            maxlongitude=maxlongitude,)
restrictions = Restrictions(
         # Get data from 5 minutes before the event to one hour after the
         # event.
         starttime=UTCDateTime(2012, 1, 1),
         endtime=UTCDateTime(2012, 5, 2),
         # You might not want to deal with gaps in the data.
         reject_channels_with_gaps=True,
         # And you might only want waveforms that have data for at least
         # 95 % of the requested time span.
         minimum_length=0.95,
         # No two stations should be closer than 10 km to each other.
         minimum_interstation_distance_in_m=10E3,
         # Only HH or BH channels. If a station has HH channels,
         # those will be downloaded, otherwise the BH. Nothing will be
         # downloaded if it has neither.
         channel_priorities=["HH[ZNE12]", "BH[ZNE]"],
         # Location codes are arbitrary and there is no rule as to which
         # location is best.
         location_priorities=["", "00", "10"],
    )
# client = Client('KNMI')
mdl = MassDownloader(providers=['KNMI','IRIS'])
mdl.download(domain=domain,
             restrictions=restrictions,
             mseed_storage='./',
             stationxml_storage='./',
             )

<span style="color:#B03052; font-size:20px"> For noisy data, there are some differences in the parameter inputs